# Monte Carlo Method : Action - Value estimation

**공부할 내용**
1. state, action, reward 가 무엇인가요?
2. Return이 무엇인가요?
3. State-Value Function 이란?
4. Action-Value Function 이란?
5. Policy 란?

**사용할 환경 : 자주 만날 Frozen Lake**

![좋은거](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSKrspq5EvLOC2Rv8dfsw1OVqD1WjN5YsmgAHVCVn3x8zLhM5Y8&usqp=CAU)


# 필요 라이브러리 불러오기.

In [ ]:
!pip install gym

In [ ]:
import numpy as np
import gym

In [ ]:
# gym.envs.registration.register(
#     id="FrozenLake-v3", entry_point='gym.envs.toy_text:FrozenLakeEnv',
#     kwargs={'map_name': '4x4', 'is_slippery': False}
# )

gym.envs.registration.register(
    id="FrozenLake-v8", entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '8x8', 'is_slippery': True}
)

In [ ]:
# env = gym.make('FrozenLake-v3')
env = gym.make('FrozenLake-v8')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

observation space: Discrete(64)
action space: Discrete(4)


## Total 연습!

1. 총 10000번의 episode를 시행한다!
    1. 하나의 episode가 진행 중일 땐,
        * states = [] # state 기록!
        * actions = [] # action 기록
        * rewards = [] # reward 기록!
    2. 하나의 episode가 종료되면
        * returns = [] # $G_t$ 제작, 기록!
        * (사실 이 단계에서 $V(s)$를 업데이트 해야 좋음 - episode마다 업데이트가 되니까!)
2. episode가 끝나면
    * sa_n : state-action별 방문횟수 업데이트 하는 넘파이 어레이 제작
    * Q : state-action별 가치값이 저장될 0으로 가득찬 2차원 array제작
    * states, sa_n, returns 이용하여 V업데이트!

$$Q(S_t, A_t) = Q(S_t, A_t) + {1\over{N(S_t,A_t)}}(G_t - Q(S_t, A_t))$$

In [ ]:
####### Your Code Here ##########

states = []
actions = []
returns = []

gamma = 0.9

for i in range(10000) :
    # 환경을 초기화 하며 초기 state 를 s0에 선언하시오.
    # rewards 와 temp_returns 에 빈 리스트를 선언하시오.
    s0 = env.reset()
    rewards = []
    temp_returns = []

    while True :
        # 랜덤한 액션을 하나 선택하여 a0에 담으시오.
        a0 = env.action_space.sample()

        # a0를 이용하여 환경과 상호작용 하면서, s1, r1, done, _ 를 선언하시오.
        s1, r1, done, _ = env.step(a0)
        
        # states에 s0을 담고 rewards에 r1을 담으시오.
        # a0 도 actions에 담으시오.
        states.append(s0)
        rewards.append(r1)
        actions.append(a0)

        if done == True :
            env.close() # 환경닫기
            break
        # 다음 단계의 s0에 s1의 값을 옮겨 선언하시오.
        s0 = s1

    ### 이 episode에 대해서 Return을 계산하여 temp_returns에 담고
    ### returns = returns + temp_returns
    ### temp_returns.append()아님 주의! insert!
    Gt = 0
    for r in rewards[::-1]:
      Gt = r + gamma * Gt
      temp_returns.insert(0, Gt)

    returns = returns + temp_returns


In [ ]:
len(states), len(returns)

(314538, 314538)

In [ ]:
# state 확인
np.unique(states)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 36,
       37, 38, 39, 40, 43, 44, 45, 47, 48, 50, 51, 53, 55, 56, 57, 58, 60,
       61, 62])

In [ ]:
sa_n = np.zeros(shape=(64,4))    # state-action 쌍이 등장한 횟수를 카운트할 2차원 어레이
Q = np.zeros(shape=(64,4))       # state-action의 가치를 기록할 2차원 어레이
# state, returns, actions 이용하여 Q를 업데이트 하자.
# Q(s,a) = E [ Gt | St=s, At=a ], Q(s,a) <- Q(s,a) + 1/n(Qt - Q(s,a))
for t, state in enumerate(states):
  At = actions[t]
  St = state
  sa_n[St][At] = sa_n[St][At] + 1
  n_sa = sa_n[St][At]
  Q[St][At] = Q[St][At] + (1/n_sa) * (returns[t] - Q[St][At])


# 자! Action-Value 바탕으로 움직인다면?

바꿔 말해, Policy를 제작한다면?

$$\pi'(s) = argmax_{a}(Q(s,a) )$$

In [ ]:
print(Q[4])
print(Q[4].argmax())

[2.33103548e-05 2.46918882e-04 4.72409927e-04 4.32512108e-04]
2


In [ ]:
s0 = env.reset()
env.render()
for i in range(1000):
    a0 = Q[s0].argmax()
    s1, r1, done, _ = env.step(a0)
    env.render()
    if done == True :
        env.close()
        break
    s0 = s1


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF

# Action-Value로 움직여도 아직 Goal로 가지 못하네...
> 반복횟수 50000, 100,000, 200,000 으로 해도 안되네...ㅠㅠ (20만하면 2시간 정도 걸림). 왜 안될까?

In [ ]:
# 반복횟수를 늘려준다. 많이

%%time

states = []
actions = []
returns = []

gamma = 0.9

for i in range(500000) :
    # 환경을 초기화 하며 초기 state 를 s0에 선언하시오.
    # rewards 와 temp_returns 에 빈 리스트를 선언하시오.
    s0 = env.reset()
    rewards = []
    temp_returns = []

    while True :
        # 랜덤한 액션을 하나 선택하여 a0에 담으시오.
        a0 = env.action_space.sample()

        # a0를 이용하여 환경과 상호작용 하면서, s1, r1, done, _ 를 선언하시오.
        s1, r1, done, _ = env.step(a0)
        
        # states에 s0을 담고 rewards에 r1을 담으시오.
        # a0 도 actions에 담으시오.
        states.append(s0)
        rewards.append(r1)
        actions.append(a0)

        if done == True :
            env.close() # 환경닫기
            break
        # 다음 단계의 s0에 s1의 값을 옮겨 선언하시오.
        s0 = s1

    ### 이 episode에 대해서 Return을 계산하여 temp_returns에 담고
    ### returns = returns + temp_returns
    ### temp_returns.append()아님 주의! insert!
    Gt = 0
    for r in rewards[::-1]:
      Gt = r + gamma * Gt
      temp_returns.insert(0, Gt)

    returns = returns + temp_returns


CPU times: user 1h 52min 12s, sys: 1min 20s, total: 1h 53min 32s
Wall time: 1h 53min 34s


In [ ]:
sa_n = np.zeros(shape=(64,4))    # state-action 쌍이 등장한 횟수를 카운트할 2차원 어레이
Q = np.zeros(shape=(64,4))       # state-action의 가치를 기록할 2차원 어레이
# state, returns, actions 이용하여 Q를 업데이트 하자.
# Q(s,a) = E [ Gt | St=s, At=a ], Q(s,a) <- Q(s,a) + 1/n(Qt - Q(s,a))
for t, state in enumerate(states):
  At = actions[t]
  St = state
  sa_n[St][At] = sa_n[St][At] + 1
  n_sa = sa_n[St][At]
  Q[St][At] = Q[St][At] + (1/n_sa) * (returns[t] - Q[St][At])


In [ ]:
print(Q[4])
print(Q[4].argmax())

[0.00039006 0.00048956 0.0006205  0.00049665]
2


In [ ]:
s0 = env.reset()
env.render()
for i in range(1000):
    a0 = Q[s0].argmax()
    s1, r1, done, _ = env.step(a0)
    env.render()
    if done == True :
        env.close()
        break
    s0 = s1


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFH